# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120739e+02     1.750633e+00
 * time: 0.06431984901428223
     1     1.061283e+01     9.565552e-01
 * time: 1.3136239051818848
     2    -1.162889e+01     1.115441e+00
 * time: 1.433763027191162
     3    -3.444247e+01     8.400461e-01
 * time: 1.5292809009552002
     4    -4.782892e+01     5.967797e-01
 * time: 1.6417460441589355
     5    -5.708320e+01     1.835557e-01
 * time: 1.722822904586792
     6    -5.986504e+01     1.311104e-01
 * time: 1.7861008644104004
     7    -6.092115e+01     5.085829e-02
 * time: 1.8655588626861572
     8    -6.131774e+01     6.779462e-02
 * time: 1.9287769794464111
     9    -6.159477e+01     5.198085e-02
 * time: 1.9920918941497803
    10    -6.179993e+01     2.469601e-02
 * time: 2.055750846862793
    11    -6.197142e+01     1.937886e-02
 * time: 2.1244828701019287
    12    -6.204162e+01     1.732034e-02
 * time: 2.1868410110473633
    13    -6.211461e+01     1.701071e-02
 * time: 2.249518871307

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671053
    AtomicLocal         -18.8557693
    AtomicNonlocal      14.8522682
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336814

    total               -62.261666453156
